# Deteksi Berita Hoax
Sebuah Proyek yang bertujuan untuk mengembangkan sebuah model Machine Learning yang mampu mengidentifikasi dan membedakan antara berita palsu (hoax) dan berita yang benar dengan akurat. Model ini dibangun berdasarkan analisis data berita yang telah dikumpulkan, dengan fitur-fitur yang dapat membedakan karakteristik antara berita palsu dan berita yang benar.

## Dataset
Dataset diambil dari Kaggle "FAKE-REAL NEWS".

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
df_valid = pd.read_csv("True.csv")

In [3]:
df_hoax = pd.read_csv("Fake.csv")

In [4]:
df_valid.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_hoax.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
rows_valid = df_valid.shape
rows_hoax = df_hoax.shape
print('Jumlah baris dataset berita valid : ' + str(rows_valid) + '\nJumlah baris dataset berita hoax : ' + str(rows_hoax))

Jumlah baris dataset berita valid : (21417, 4)
Jumlah baris dataset berita hoax : (23481, 4)


In [7]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [8]:
df_hoax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


### Data Validation (Missing Values)

In [9]:
df_valid.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [10]:
df_hoax.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

### Data Validation (Duplicated)

In [11]:
duplikat_valid = df_valid.duplicated().sum()
duplikat_hoax = df_hoax.duplicated().sum()
print('Jumlah data duplikat berita valid : ' + str(duplikat_valid) + '\nJumlah data duplikat berita hoax : ' + str(duplikat_hoax))

Jumlah data duplikat berita valid : 206
Jumlah data duplikat berita hoax : 3


In [12]:
duplicate_mask = df_valid.duplicated(keep=False)
duplicates = df_valid[duplicate_mask]
print(duplicates)

                                                   title  \
416    Senate tax bill stalls on deficit-focused 'tri...   
445    Senate tax bill stalls on deficit-focused 'tri...   
762    Trump warns 'rogue regime' North Korea of grav...   
778    Trump warns 'rogue regime' North Korea of grav...   
850    Republicans unveil tax cut bill, but the hard ...   
...                                                  ...   
21290  Europeans, Africans agree renewed push to tack...   
21345  Thailand's ousted PM Yingluck has fled abroad:...   
21353  Thailand's ousted PM Yingluck has fled abroad:...   
21406  U.S., North Korea clash at U.N. forum over nuc...   
21408  U.S., North Korea clash at U.N. forum over nuc...   

                                                    text       subject  \
416    WASHINGTON (Reuters) - The U.S. Senate on Thur...  politicsNews   
445    WASHINGTON (Reuters) - The U.S. Senate on Thur...  politicsNews   
762    BEIJING (Reuters) - U.S. President Donald Trum... 

Setalah diperiksa langsung di dalam datasetnya, terbukti jika baris dari Indeks 416-21408 memiliki data yang sama.

In [13]:
#Menghapus data duplikat
df_valid = df_valid.drop_duplicates()

In [14]:
df_valid.duplicated().sum()

0

In [15]:
# Pengecekan data duplikat df_hoax
duplicate_mask = df_hoax.duplicated(keep=False)
duplicates = df_hoax[duplicate_mask]
print(duplicates)

                                                   title  \
9941   HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...   
9942   HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...   
11445  FORMER DEMOCRAT WARNS Young Americans: “Rioter...   
11446  FORMER DEMOCRAT WARNS Young Americans: “Rioter...   
14924  [VIDEO] #BlackLivesMatter Terrorists Storm Dar...   
14925  [VIDEO] #BlackLivesMatter Terrorists Storm Dar...   

                                                    text   subject  \
9941   No time to waste   we've got to fight with eve...  politics   
9942   No time to waste   we've got to fight with eve...  politics   
11445   Who is silencing political speech, physically...  politics   
11446   Who is silencing political speech, physically...  politics   
14924  They were probably just looking for a  safe sp...  politics   
14925  They were probably just looking for a  safe sp...  politics   

               date  
9941    Sep 9, 2017  
9942    Sep 9, 2017  
11445  Mar 10, 2017  


In [16]:
#Menghapus data duplikat
df_hoax = df_hoax.drop_duplicates()

In [17]:
duplikat_valid = df_valid.duplicated().sum()
duplikat_hoax = df_hoax.duplicated().sum()
print('Jumlah data duplikat berita valid : ' + str(duplikat_valid) + '\nJumlah data duplikat berita hoax : ' + str(duplikat_hoax))

Jumlah data duplikat berita valid : 0
Jumlah data duplikat berita hoax : 0


### Data Transform

Deklarasi kolom atau fitur target

In [18]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21211 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21211 non-null  object
 1   text     21211 non-null  object
 2   subject  21211 non-null  object
 3   date     21211 non-null  object
dtypes: object(4)
memory usage: 828.6+ KB


In [19]:
df_valid.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [20]:
df_hoax["Tag"] = 0
df_valid["Tag"] = 1

In [21]:
df_valid.head()

,title,text,subject,date,Tag
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [22]:
df_hoax.head()

,title,text,subject,date,Tag
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [23]:
# Memisahkan 10 baris dari setiap dataset untuk digunakan sebagai manual testing
df_valid.shape, df_hoax.shape

((21211, 5), (23478, 5))

In [24]:
df_valid_manual_testing = df_valid.tail(10)
for i in range(21211,21201,-1):
    df_valid.drop([i], axis = 0, inplace = True)
    
    
df_hoax_manual_testing = df_hoax.tail(10)
for i in range(23478,23468,-1):
    df_hoax.drop([i], axis = 0, inplace = True)

In [25]:
df_hoax_manual_testing.head()

,title,text,subject,date,Tag
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [26]:
df_valid_manual_testing.head()

,title,text,subject,date,Tag
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1


In [27]:
#Eksport dataset manual-testing.csv
df_manual_testing = pd.concat([df_valid_manual_testing, df_hoax_manual_testing], axis=0)
df_manual_testing.to_csv("manual-testing.csv")

### Concat Dataframe

In [28]:
df = pd.concat([df_valid, df_hoax], axis=0)

In [29]:
df.head()

,title,text,subject,date,Tag
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [30]:
df.shape

(44669, 5)

In [31]:
df.columns

Index(['title', 'text', 'subject', 'date', 'Tag'], dtype='object')

### Penghapusan Kolom yang tidak digunakan

In [32]:
df = df.drop(["title", "subject", "date"], axis = 1)

In [33]:
df.shape

(44669, 2)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44669 entries, 0 to 23480
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44669 non-null  object
 1   Tag     44669 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [35]:
df = df.rename(columns={'Tag' : 'tag'})

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44669 entries, 0 to 23480
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44669 non-null  object
 1   tag     44669 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [37]:
df.tail()

,text,tag
23466,21st Century Wire says For those who still ref...,0
23467,"21st Century Wire says So far, after nearly 20...",0
23468,21st Century Wire says If you ve been followin...,0
23479,21st Century Wire says Al Jazeera America will...,0
23480,21st Century Wire says As 21WIRE predicted in ...,0


In [38]:
df.head()

,text,tag
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


Bisa dilihat kalau posisi atau urutan berita real dengan berita hoax saling beu-urutan, untuk mencegah Mechine Learning overvitting maka kita akan melakukan Random Shufflineg agar datanya tidak saling berurutan.

### Random Shuffling

In [39]:
df = df.sample(frac = 1)

In [40]:
df.head()

,text,tag
5209,WASHINGTON (Reuters) - The Senate on Wednesday...,1
5767,"On Wednesday, the Democratic party staged a po...",0
5228,WASHINGTON (Reuters) - The Republican chairman...,1
7866,PARIS (Reuters) - Europe should challenge the ...,1
8030,WASHINGTON (Reuters) - Democrat Hillary Clinto...,1


In [41]:
df.tail()

,text,tag
13686,"STAVROPOL, Russia (Reuters) - President Vladim...",1
14030,When you hear the left screaming about hatefu...,0
15506,LONDON (Reuters) - Iran s continued detention ...,1
571,While Donald Trump and his brownshirts are bus...,0
12927,Hillary Clinton is preparing to get back on th...,0


In [42]:
df.columns

Index(['text', 'tag'], dtype='object')

In [43]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [44]:
df.head()

,text,tag
0,WASHINGTON (Reuters) - The Senate on Wednesday...,1
1,"On Wednesday, the Democratic party staged a po...",0
2,WASHINGTON (Reuters) - The Republican chairman...,1
3,PARIS (Reuters) - Europe should challenge the ...,1
4,WASHINGTON (Reuters) - Democrat Hillary Clinto...,1


### Wordopt

In [45]:
def wordopt(text) :
    text = text.lower()
    text = re.sub("\\W"," ", text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [46]:
df["text"] = df["text"].apply(wordopt)

### Deklarasi Independen & Dependen Variabel

In [47]:
x = df["text"]
y = df["tag"]

### Data Splitting
- Data Train = 75%
- Data Test = 25%

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

### Text -> Vector

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer()
xv_train = vector.fit_transform(x_train)
xv_test = vector.transform(x_test)

### Modelling

### Decision Tree Classication

In [50]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [51]:
pred_dt = DT.predict(xv_test)

In [52]:
DT.score(xv_test, y_test)

0.9946275071633238

In [53]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5887
           1       1.00      0.99      0.99      5281

    accuracy                           0.99     11168
   macro avg       0.99      0.99      0.99     11168
weighted avg       0.99      0.99      0.99     11168



### Gradient Boosting Classifier

In [54]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [55]:
pred_gbc = GBC.predict(xv_test)

In [56]:
GBC.score(xv_test, y_test)

0.9951647564469914

In [57]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5887
           1       0.99      1.00      0.99      5281

    accuracy                           1.00     11168
   macro avg       1.00      1.00      1.00     11168
weighted avg       1.00      1.00      1.00     11168



### Logistic Regression

In [58]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [59]:
pred_lr=LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9881805157593123

In [60]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5887
           1       0.98      0.99      0.99      5281

    accuracy                           0.99     11168
   macro avg       0.99      0.99      0.99     11168
weighted avg       0.99      0.99      0.99     11168



### Random Forest Classifier

In [61]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [62]:
pred_rfc = RFC.predict(xv_test)
RFC.score(xv_test, y_test)

0.9897922636103151

In [63]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5887
           1       0.99      0.99      0.99      5281

    accuracy                           0.99     11168
   macro avg       0.99      0.99      0.99     11168
weighted avg       0.99      0.99      0.99     11168



### Linear SVC

In [64]:
from sklearn.svm import LinearSVC

SVC = LinearSVC(dual=False)
SVC.fit(xv_train, y_train)

LinearSVC(dual=False)

In [65]:
pred_svc = SVC.predict(xv_test)
SVC.score(xv_test, y_test)

0.9944484240687679

In [66]:
print(classification_report(y_test, pred_svc))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5887
           1       0.99      1.00      0.99      5281

    accuracy                           0.99     11168
   macro avg       0.99      0.99      0.99     11168
weighted avg       0.99      0.99      0.99     11168



### Testing Model

In [73]:
def output_lable(n):
    if n == 0:
        return "Berita Hoax"
    elif n == 1:
        return "Berita Real"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vector.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_SVC = SVC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nSVC Prediction: {} \nRFC Prediction: {}".
                 format(output_lable(pred_LR[0]),
                    output_lable(pred_DT[0]), 
                    output_lable(pred_GBC[0]),
                    output_lable(pred_SVC[0]),
                    output_lable(pred_RFC[0])))

In [74]:
news = str(input())
manual_testing(news)

 SAO PAULO (Reuters) - Cesar Mata Pires, the owner and co-founder of Brazilian engineering conglomerate OAS SA, one of the largest companies involved in Brazil s corruption scandal, died on Tuesday. He was 68. Mata Pires died of a heart attack while taking a morning walk in an upscale district of S o Paulo, where OAS is based, a person with direct knowledge of the matter said. Efforts to contact his family were unsuccessful. OAS declined to comment. The son of a wealthy cattle rancher in the northeastern state of Bahia, Mata Pires  links to politicians were central to the expansion of OAS, which became Brazil s No. 4 builder earlier this decade, people familiar with his career told Reuters last year. His big break came when he befriended Antonio Carlos Magalh es, a popular politician who was Bahia governor several times, and eventually married his daughter Tereza. Brazilians joked that OAS stood for  Obras Arranjadas pelo Sogro  - or  Work Arranged by the Father-In-Law.   After years o



LR Prediction: Berita Real 
DT Prediction: Berita Real 
GBC Prediction: Berita Real 
SVC Prediction: Berita Real 
RFC Prediction: Berita Real


In [75]:
news = str(input())
manual_testing(news)

 Paul Craig RobertsIn the last years of the 20th century fraud entered US foreign policy in a new way.  On false pretenses Washington dismantled Yugoslavia and Serbia in order to advance an undeclared agenda. In the 21st century this fraud multiplied many times. Afghanistan, Iraq, Somalia, and Libya were destroyed, and Iran and Syria would also have been destroyed if the President of Russia had not prevented it.  Washington is also behind the current destruction of Yemen, and Washington has enabled and financed the Israeli destruction of Palestine.  Additionally, Washington operated militarily within Pakistan without declaring war, murdering many women, children, and village elders under the guise of  combating terrorism.  Washington s war crimes rival those of any country in history.I have documented these crimes in my columns and books (Clarity Press). Anyone who still believes in the purity of Washington s foreign policy is a lost soul  Russia and China now have a strategic alliance



LR Prediction: Berita Hoax 
DT Prediction: Berita Hoax 
GBC Prediction: Berita Hoax 
SVC Prediction: Berita Hoax 
RFC Prediction: Berita Hoax


In [76]:
news = str(input())
manual_testing(news)

 21st Century Wire says This week, the historic international Iranian Nuclear Deal was punctuated by a two-way prisoner swap between Washington and Tehran, but it didn t end quite the way everyone expected. On the Iranian side, one of the U.S. citizens who was detained in Iran, Nosratollah Khosravi-Roodsari, has stayed in Iran, but on the U.S. side   all 7 of the Iranians held in U.S. prisons DID NOT show up to their flight to Geneva for the prisoner exchange   with at least 3 electing to stay in the U.S  TEHRAN SIDE: In Iran, 5 U.S. prisoners were released, with 4 of them making their way to Germany via Switzerland.Will Robinson Daily MailNone of the Iranians freed in the prisoner swap have returned home and could still be in the United States, it has been reported.The seven former inmates, who were released as part of a deal with the Islamic republic, did not show up to get a flight to Geneva, Switzerland, where the exchange was set to take place on Sunday.Three of the Iranians have 



LR Prediction: Berita Hoax 
DT Prediction: Berita Hoax 
GBC Prediction: Berita Hoax 
SVC Prediction: Berita Hoax 
RFC Prediction: Berita Hoax


In [77]:
news = str(input())
manual_testing(news)

 Indonesia Discovers Hidden Treasure Trove in Bali. In a groundbreaking discovery, Indonesian archaeologists have unearthed a hidden treasure trove in the heart of Bali. The treasure is said to contain untold riches, including gold, jewels, and ancient artifacts, believed to be worth billions of dollars. The find has sparked excitement and intrigue worldwide, as experts and treasure hunters flock to the island to get a glimpse of this incredible discovery.




LR Prediction: Berita Hoax 
DT Prediction: Berita Hoax 
GBC Prediction: Berita Hoax 
SVC Prediction: Berita Hoax 
RFC Prediction: Berita Hoax


In [81]:
news = str(input())
manual_testing(news)

 UNITED NATIONS (Reuters) - Two North Korean shipments to a Syrian government agency responsible for the country s chemical weapons program were intercepted in the past six months, according to a confidential United Nations report on North Korea sanctions violations. The report by a panel of independent U.N. experts, which was submitted to the U.N. Security Council earlier this month and seen by Reuters on Monday, gave no details on when or where the interdictions occurred or what the shipments contained.   The panel is investigating reported prohibited chemical, ballistic missile and conventional arms cooperation between Syria and the DPRK (North Korea),  the experts wrote in the 37-page report.   Two member states interdicted shipments destined for Syria. Another Member state informed the panel that it had reasons to believe that the goods were part of a KOMID contract with Syria,  according to the report. KOMID is the Korea Mining Development Trading Corporation. It was blacklisted 



LR Prediction: Berita Real 
DT Prediction: Berita Real 
GBC Prediction: Berita Real 
SVC Prediction: Berita Real 
RFC Prediction: Berita Real


In [83]:
news = str(input())
manual_testing(news)

 WASHINGTON (Reuters) - Bali, often referred to as the 'Island of the Gods,' is a picturesque destination known for its stunning natural landscapes, vibrant culture, and warm hospitality. Situated in Indonesia, this tropical paradise is famous for its lush rice terraces, pristine beaches, and vibrant coral reefs.  Bali offers a diverse range of experiences for travelers. Visitors can explore ancient temples like Uluwatu and Tanah Lot, where they can witness traditional Balinese rituals and breathtaking sunsets. The island is also a haven for adventure seekers, offering activities such as surfing, snorkeling, and hiking in its lush jungles.  The local cuisine in Bali is a culinary delight, with an array of flavors and dishes to explore. Balinese dance and music performances showcase the rich cultural heritage of the island, and tourists are often welcomed to join in the festivities.  In addition to its natural beauty and cultural richness, Bali is known for its warm and friendly locals 



LR Prediction: Berita Hoax 
DT Prediction: Berita Real 
GBC Prediction: Berita Real 
SVC Prediction: Berita Real 
RFC Prediction: Berita Hoax


In [85]:
news = str(input())
manual_testing(news)

 WASHINGTON (Reuters) - Donald Trump is the former president of the United States




LR Prediction: Berita Real 
DT Prediction: Berita Real 
GBC Prediction: Berita Real 
SVC Prediction: Berita Real 
RFC Prediction: Berita Real


# Kesimpulan :

1. Baru saya sadari setelah melakukan manual-testing model bahwa project mechine learning ini cenderung lebih Overvitting. Setelah penulis cek dataset yang digunakan sebagai sample dan unit test dalam project Mechine Learning ini bahwa berita-berita yang real (nyata) selalu diawali dengan " .... (Reuters) - " sedangkan untuk berita Hoax tidak diawali dengan kalimat seperti itu, sehingga setiap model yang digunakan cenderung hanya menghafal pola kalimat berita valid dan hoax.
2. Walaupun cenderung overvitting, namun model SVC cukup dapat membedakan mana berita real dan mana berita hoax, hal ini dibuktikan dari beberapa berita-berita yang saya uji.
3. Kualitas dataset sangat mempengaruhi kinerja model.
4. Next project adalah melakukan cleaning data " .... (Reuters) - " yang terdapat pada dataset True.csv